In [7]:
parentfolder = 'D:\\Finn\\KPUM_NODDI\\DICOM_KPUM_NODDI'
outputdicombasefolder = 'sourcedata'

# User input
subject = '010'
session = 'MR1'
inputdicomfolder = '010_MR1_8253422_20211011/DICOM_NODDI'

In [8]:
# Define input and output parameters for dcm2niix
import os, subprocess
import pydicom as pm
from glob import glob

def anonymize_dicom(in_path, out_path):
    # Anonymizes input DCM according to data_elements in list
    
    dicom_file = pm.dcmread(in_path)
    
    # DCM tags to anonymize
    data_elements = ['PatientName',
                     'PatientBirthDate',
                     'PatientID']
    for de in data_elements:
        if de in dicom_file:
            dicom_file.data_element(de).value = 'Anonymous'
            
    dicom_file.save_as(out_path)


# Define paths
inputdicomdir = os.path.join(parentfolder, inputdicomfolder)
outputdicomdir = os.path.join(parentfolder, outputdicombasefolder, f'sub-{subject}', f'ses-{session}')
if not os.path.exists(outputdicomdir): # then make this directory
    os.makedirs(outputdicomdir)

# Re-organize using dcm2niix
p=subprocess.Popen(f"dcm2niix -d 8 -b o -r y -w 1 -o {outputdicomdir} -f s%2s_%p\\%p_%5r.dcm {inputdicomdir}", stdout=subprocess.PIPE, shell=True)

# and now anonymize this
in_folder=os.listdir(outputdicomdir)
for folder in in_folder:
    in_slices = glob(os.path.join(outputdicomdir, folder, '*.dcm'))
    #print(in_slices)
    for in_slice_ in in_slices:
        out_slice_ = in_slice_ # write to same file = overwrite with anonymized version
        anonymize_dicom(in_slice_, out_slice_) # write to file defined by out_slice 


c:\Users\finnl\miniconda3\envs\kpum_noddi\lib\site-packages\pydicom\valuerep.py:290: UserWarning: Invalid value for VR DA: 'Anonymous'.
  warnings.warn(msg)
